In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

In [3]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []

    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
    
        print(label, "completed")
    return image_paths, labels

In [25]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [26]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [27]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [28]:
print(test)

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [29]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = "grayscale" )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48,48,1)
    return features

In [30]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [31]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [32]:
x_train = train_features/255.0
x_test = test_features/255.0

In [33]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [44]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [45]:
y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test,num_classes=7)

In [46]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [47]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 [==============================] - 269s 1s/step - loss: 1.8226 - accuracy: 0.2445 - val_loss: 1.7968 - val_accuracy: 0.2629
Epoch 2/100
226/226 [==============================] - 257s 1s/step - loss: 1.7855 - accuracy: 0.2593 - val_loss: 1.7051 - val_accuracy: 0.2972
Epoch 3/100
226/226 [==============================] - 257s 1s/step - loss: 1.6851 - accuracy: 0.3144 - val_loss: 1.5739 - val_accuracy: 0.3900
Epoch 4/100
226/226 [==============================] - 257s 1s/step - loss: 1.5601 - accuracy: 0.3859 - val_loss: 1.4502 - val_accuracy: 0.4461
Epoch 5/100
226/226 [==============================] - 254s 1s/step - loss: 1.4790 - accuracy: 0.4275 - val_loss: 1.3513 - val_accuracy: 0.4687
Epoch 6/100
226/226 [==============================] - 237s 1s/step - loss: 1.4190 - accuracy: 0.4530 - val_loss: 1.2937 - val_accuracy: 0.5081
Epoch 7/100
226/226 [==============================] - 222s 982ms/step - loss: 1.3776 - accuracy: 0.4665 - val_loss: 1.2619 - val_accura

In [52]:
model_json = model.to_json()
with open ("emotiondetector.json","w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

c:\Users\Sandeep\Desktop\fed\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
from keras.models import model_from_json

In [54]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [55]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [56]:
def ef(image):
    img = load_img(image,color_mode = "grayscale" )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [58]:
image = 'images/train/angry/10207.jpg'
print("Original image is of angry")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

Original image is of angry
1/1 [==============================] - 0s 75ms/step
model prediction is  angry


In [60]:
import matplotlib.pyplot as plt 
%matplotlib inline